# Week 4 Programming Assignment
## Imports

In [1]:
from random import randint, uniform

from pulp import *  # pyright: ignore [reportWildcardImportFromLibrary]

## Problem 1

In [2]:
def k_tsp_mtz_encoding(n, k, cost_matrix):
    # Check inputs are OK
    assert 1 <= k < n
    assert len(cost_matrix) == n, f"Cost matrix is not {n}x{n}"
    assert all(len(row) == n for row in cost_matrix), f"Cost matrix is not {n}x{n}"

    # Create decision variables
    # Binary variables
    binary_vars = [
        [LpVariable(f"x_{{{i},{j}}}", cat=LpBinary) if i != j else None for j in range(n)] for i in range(n)
    ]
    # Time stamp variables
    time_stamps = [LpVariable(f"t_{i}", lowBound=0, upBound=n) for i in range(1, n)]

    # Create the problem
    prob = LpProblem("kTSP", LpMinimize)

    # Add the objective function
    prob += lpSum(
        lpSum(cost_matrix[i][j] * binary_vars[i][j] for j in filter(lambda x: x != i, range(n)))
        for i in range(n)
    )

    # Add the degree constraints
    # Vertex 0
    num_enter = lpSum(binary_vars[j][0] for j in range(1, n))
    num_leave = lpSum(binary_vars[0][j] for j in range(1, n))
    prob += num_enter == k
    prob += num_leave == k

    # All the other vertices
    for i in range(1, n):
        num_enter = lpSum(binary_vars[j][i] for j in filter(lambda x: x != i, range(n)))
        num_leave = lpSum(binary_vars[i][j] for j in filter(lambda x: x != i, range(n)))
        prob += num_enter == 1
        prob += num_leave == 1

    # Add time stamp constraints
    for i in range(1, n):
        for j in filter(lambda x: x != i, range(1, n)):
            x = binary_vars[i][j]
            assert x is not None
            prob += time_stamps[j - 1] >= time_stamps[i - 1] + x - (1 - x) * (n + 1)

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False))
    status = LpStatus[prob.status]
    assert status == "Optimal", f"Unexpected non-optimal status: {status}"

    # Extract tours
    tours = []

    # Get all second vertices
    second = [j for j, x in enumerate(binary_vars[0]) if x is not None and x.varValue > 0]  # pyright: ignore [reportOptionalOperand]
    assert len(second) == k, "Could not find second vertex for each salesman"

    for v in second:
        tour = [0, v]

        while True:
            nxt = [j for j, x in enumerate(binary_vars[tour[-1]]) if x is not None and x.varValue > 0]  # pyright: ignore [reportOptionalOperand]
            assert len(nxt) == 1
            if nxt[0] == 0:
                break
            tour.append(nxt[0])

        tours.append(tour)

    return tours

In [3]:
n = 5
k = 2
cost_matrix = [
    [None, 3, 4, 3, 5],
    [1, None, 2, 4, 1],
    [2, 1, None, 5, 4],
    [1, 1, 5, None, 4],
    [2, 1, 3, 5, None],
]
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f"Your code returned tours: {all_tours}")
assert len(all_tours) == k, f"k={k} must yield {k} tours -- your code returns {len(all_tours)} tours instead"

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, "Each salesperson tour must start from vertex 0"
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
print(f"Tour cost obtained by your code: {tour_cost}")
assert abs(tour_cost - 12) <= 0.001, f"Expected tour cost is 12, your code returned {tour_cost}"

for i in range(1, n):
    is_in_tour = [i in tour for tour in all_tours]
    assert sum(is_in_tour) == 1, f"Vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect"

print("Test passed: 3 points")

Your code returned tours: [[0, 2, 1, 4], [0, 3]]
Tour cost obtained by your code: 12
Test passed: 3 points


In [4]:
n = 5
k = 3
cost_matrix = [
    [None, 3, 4, 3, 5],
    [1, None, 2, 4, 1],
    [2, 1, None, 5, 4],
    [1, 1, 5, None, 4],
    [2, 1, 3, 5, None],
]
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f"Your code returned tours: {all_tours}")
assert len(all_tours) == k, f"k={k} must yield {k} tours -- your code returns {len(all_tours)} tours instead"

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, "Each salesperson tour must start from vertex 0"
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
print(f"Tour cost obtained by your code: {tour_cost}")
assert abs(tour_cost - 16) <= 0.001, f"Expected tour cost is 16, your code returned {tour_cost}"

for i in range(1, n):
    is_in_tour = [i in tour for tour in all_tours]
    assert sum(is_in_tour) == 1, f"Vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect"

print("Test passed: 2 points")

Your code returned tours: [[0, 1, 4], [0, 2], [0, 3]]
Tour cost obtained by your code: 16
Test passed: 2 points


In [5]:
n = 8
k = 2
cost_matrix = [
    [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0, 1, 2, 1, 1, None, 1, 1],
    [1, 0, 1, 2, 2, 2, None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f"Your code returned tours: {all_tours}")
assert len(all_tours) == k, f"k={k} must yield {k} tours -- your code returns {len(all_tours)} tours instead"

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, "Each salesperson tour must start from vertex 0"
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
print(f"Tour cost obtained by your code: {tour_cost}")
assert abs(tour_cost - 4) <= 0.001, f"Expected tour cost is 4, your code returned {tour_cost}"

for i in range(1, n):
    is_in_tour = [i in tour for tour in all_tours]
    assert sum(is_in_tour) == 1, f"Vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect"

print("Test passed: 3 points")

Your code returned tours: [[0, 6, 2, 1], [0, 7, 3, 4, 5]]
Tour cost obtained by your code: 4
Test passed: 3 points


In [6]:
n = 8
k = 4
cost_matrix = [
    [None, 1, 1, 1, 1, 1, 1, 1],
    [0, None, 1, 2, 1, 1, 1, 1],
    [1, 0, None, 1, 2, 2, 2, 1],
    [1, 2, 2, None, 0, 1, 2, 1],
    [1, 1, 1, 1, None, 1, 1, 1],
    [0, 1, 2, 1, 1, None, 1, 1],
    [1, 0, 1, 2, 2, 2, None, 1],
    [1, 2, 2, 0, 1, 2, 1, None],
]
all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
print(f"Your code returned tours: {all_tours}")
assert len(all_tours) == k, f"k={k} must yield {k} tours -- your code returns {len(all_tours)} tours instead"

tour_cost = 0
for tour in all_tours:
    assert tour[0] == 0, "Each salesperson tour must start from vertex 0"
    i = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
print(f"Tour cost obtained by your code: {tour_cost}")
assert abs(tour_cost - 6) <= 0.001, f"Expected tour cost is 6, your code returned {tour_cost}"

for i in range(1, n):
    is_in_tour = [i in tour for tour in all_tours]
    assert sum(is_in_tour) == 1, f"Vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect"

print("Test passed: 2 points")

Your code returned tours: [[0, 2], [0, 5], [0, 6, 1], [0, 7, 3, 4]]
Tour cost obtained by your code: 6
Test passed: 2 points


In [7]:
def create_cost(n):
    return [[uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]


for trial in range(5):
    print(f"Trial #{trial}")

    n = randint(5, 11)
    k = randint(2, n // 2)
    print(f"n={n}, k={k}")

    cost_matrix = create_cost(n)
    print("cost_matrix =")
    print(cost_matrix)

    all_tours = k_tsp_mtz_encoding(n, k, cost_matrix)
    print(f"Your code returned tours: {all_tours}")
    assert (
        len(all_tours) == k
    ), f"k={k} must yield {k} tours -- your code returns {len(all_tours)} tours instead"

    tour_cost = 0
    for tour in all_tours:
        assert tour[0] == 0, "Each salesperson tour must start from vertex 0"
        i = 0
        for j in tour[1:]:
            tour_cost += cost_matrix[i][j]
            i = j
        tour_cost += cost_matrix[i][0]  # pyright: ignore
    print(f"Tour cost obtained by your code: {tour_cost}")

    for i in range(1, n):
        is_in_tour = [i in tour for tour in all_tours]
        assert sum(is_in_tour) == 1, f"Vertex {i} is in {sum(is_in_tour)} tours -- this is incorrect"

    print("----------")

print("Test passed: 15 points")

Trial #0
n=11, k=5
cost_matrix =
[[None, 3.3364268898548053, 0.03945890897336379, 4.4943165570239705, 4.370209515648252, 3.5486427703103907, 3.6708041065005514, 1.7234714755018399, 1.32001970456618, 3.2692158486565064, 2.0278722917082845], [3.1385023302234356, None, 0.43338877828105915, 3.350181669196241, 2.8491857082490784, 2.75423125555081, 4.790092051928455, 1.3543658433589556, 2.6148016387232897, 2.2938020234750276, 0.2516208947110138], [2.593532974384573, 2.2148325480157287, None, 4.339584843922656, 0.0670736475723982, 0.09490021622501121, 4.018042190752147, 1.002488651839053, 3.24116237550005, 4.386182663212393, 3.610736980038563], [1.8055891947606555, 4.064053825720557, 3.0357153174887745, None, 2.47120410848885, 0.025883613116524096, 2.644434810444901, 0.5628819002028201, 2.975472785276386, 4.150932182447537, 3.6741803421340524], [1.2558404322547079, 3.3146543118235483, 0.8532690019032113, 2.386274442825474, None, 0.00827641841323834, 2.2816012995524555, 2.3972562338002406, 1.7